In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers import Input


img_width, img_height = 100, 100
train_data_dir = "data/train"
validation_data_dir = "data/val"
nb_train_samples = 326
nb_validation_samples = 66 
batch_size = 5
epochs = 50

### Build the network 
img_input = Input(shape=(img_width, img_height, 3))
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

model = Model(inputs = img_input, outputs = x)

#Print model summary
model.summary()

#Create a dictionary with layer name
layer_dict = dict([(layer.name, layer) for layer in model.layers])
[layer.name for layer in model.layers]

#Use pretrained weights from the downloaed VGG19 model h5 file.
#Use pretrained weights from the downloaed VGG19 model h5 file.
import h5py
weights_path = 'vgg19_weights.h5' # ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5)
f = h5py.File(weights_path)
list(f.keys())
layer_names = [layer.name for layer in model.layers]

# Here we are extracting model_weights for each and every layer from the .h5 file
# With a for loop we can set_weights for the entire network.
for i in layer_dict.keys():
    if (i.startswith("input")):
        continue
    else:
        weight_names = f[i].attrs["weight_names"]
        weights = [f[i][j] for j in weight_names]
        index = layer_names.index(i)
        #print("Before" + model.layers[index].name)
        #print(model.layers[index].get_weights())
        model.layers[index].set_weights(weights)
        #print("After")
        #print(model.layers[index].get_weights())
        
        # Freeze the layers which you don't want to train. Here I am freezing all layers.
        model.layers[index].trainable = False

#Print the model so far
model.summary()

#Adding the trainable fully connected  Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

#Print the final model
model_final.summary()

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.01, momentum=0.9), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg19_2BlockTransfer.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
train_generator,
steps_per_epoch = nb_train_samples//batch_size,
epochs = epochs,
validation_data = validation_generator,
validation_steps=nb_validation_samples // batch_size,
callbacks = [checkpoint, early])



In [ ]:
#Load the model and Visualize the network
from keras.models import load_model
modelLoaded = load_model('vgg19_2BlockTransfer.h5')
from quiver_engine.server import launch
from gevent.pywsgi import WSGIServer
launch(modelLoaded, input_folder='./', port=7002)

In [ ]:
#Predict dollar value class for a sample image
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
modelLoaded = load_model('vgg19_2BlockTransfer.h5')
#load the image
imageLoaded = load_img('IMG_0878.jpg', target_size=(100, 100))
# convert the image pixels to a numpy array
imageLoaded = img_to_array(imageLoaded)
# reshape data for the model
imageLoaded = imageLoaded.reshape((1, imageLoaded.shape[0], imageLoaded.shape[1], imageLoaded.shape[2]))
# predict the probability across all output classes
prediction = modelLoaded.predict(imageLoaded)
# Print the prediction
print(prediction)